# Quantum Information and Computing
## Tensor networks as neural network layers for supervised learning

### Index

1. Introduction
2. Using tensor networks for classification tasks
    - Toy model
    - MNIST handwritten digits
    - Fruit and vegetable recognition
3. Comparison with standard neural networks
    - Fully-connected networks
    - Convolutional networks

## Introduction

# ##Placeholder for intro block##

## Using tensor networks for classification tasks

In order to simplify computational tasks, a combination of the modules ```TensorFlow``` and ```TensorNetwork``` has been used.

- [TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf), [source](https://github.com/tensorflow/tensorflow)
- [TensorNetwork documentation](https://tensornetwork.readthedocs.io/en/latest/), [source](https://github.com/google/TensorNetwork)

### Basic toy model

#### Data generation

In [12]:
from final import *
%load_ext autoreload
%autoreload 2
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
unit_sq = np.mgrid[0:1:.001, 0:1:.001]

#Initialize multivar gaussian distributions
rv_tL = mv_norm([0., 1.], [[0.1, 0], [0., 0.15]]) #top-Left gaussian
rv_bR = mv_norm([1., 0.], [[0.175, 0], [0., 0.09]]) #bottom-Right gaussian

#Generate the multivariate density functions
pos = np.dstack(unit_sq)
dist1 = rv_tL.pdf(pos)
dist2 = rv_bR.pdf(pos)

#Compute the optimal theoretical boundary
x_vals, y_est = optimal_boundary(dist1, dist2, unit_sq, tolerance=1e-4)

#Generate data samples and labels to use in the classification task.
x1, labels1, x2, labels2 = gen_samples(1000, rv_tL, rv_bR)
x, labels = np.vstack([x1,x2]), np.vstack([labels1,labels2]).flatten()


#Generate random shuffler for the data
shuffler = np.random.permutation(len(x))
x, labels = x[shuffler], labels[shuffler]
#shuffle and split data into training and test sets.
x_train, x_test, labels_train, labels_test = train_test_split(x, labels, test_size=0.2, random_state=420)

For this toy model, the objective function is chosen as the binary negative logarithm loss. Because of this, the output ought to have two components, each indicating the likelihood of predicting either label. These are obtained at the free index of the tensor network, upon application of a softmax activation on the resulting tensor components.

The model will aim to approximate the separating curve as illustrated in the plots below. This is the theoretical optimal separating plane, defined as the set of locations where the two distributions are equal $-$ where a classifier will predict either class with equal probability.

In [ ]:
toy_plot = toy_model_plot(unit_sq, dist1, dist2, x_vals, y_est, x1, x2, x_test, labels_test)

#### Feature mapping

The matrix product state will have as many nodes as feature inputs. Having two dimensional data $(x_1, x_2)$, the feature map will have rank 2:

$$
\Phi(x_1, x_2) = \phi^{s_1}(x_1) \otimes \phi^{s_2}(x_2)
$$

Where $\phi(\cdot)$ is a feature map $\phi : \mathbb{R} \rightarrow \mathbb{R^d}$ where $d$ is the number of components of each index $s_{i}$ of the input $\Phi$. The impact of this value will be studied further.

The mapping is chosen as follows:
$$
\phi(x)_{s_i} = \sqrt{\binom{d - 1}{s_i - 1}} \cos^{d - s_i}\left(\frac{\pi}{2}x \right) \sin^{s_i - 1}\left(\frac{\pi}{2}x\right)
$$

It can be promptly shown that this mapping will be normalized to unity for any value of $d$.

In [ ]:
d=3
#Feature mapping dataset
x_ftrain, y_htrain, x_ftest, y_htest = fmap(d, x_train, x_test, labels_train, labels_test)

The tensor network is defined as a ```Keras``` model of a single layer. This allows for easy implementation of different algorithms of gradient descent and the respective back-propagation.

The layer is defined as a subclass of ```tensorflow.keras.layers.Layer```, inheriting its methods and attributes. The number of nodes in the matrix product state (MPS) equals the dimensionality of the input: an $N$-dimensional input will yield a tensor of rank $N$. With two-dimensional toy data, we use two MPS tensors. The bond dimension can be tuned as a hyper-parameter of the model. A larger bond dimension will, in principle, augment the classifier's predictive power.

In [ ]:
#model training

bond_dim = 5
tnetwork = build_model(d, bond_dim, 'SGD', batch_size=None)

t_i = time.time()
history = tnetwork.fit(x_ftrain, y_htrain, batch_size=16, validation_split=0.2, epochs=100, verbose=0, shuffle=True)
t_f = time.time()
delta_t = t_f - t_i
#plotting history
history_plot = plot_loss_acc(history.history, figsize = (9, 5), tight_layout='pad')
print(f'Total training time: {delta_t:.4f} seconds over {len(history.epoch)} epochs.')




In [ ]:
m_list = [2, 5, 11]
d_list = [2, 4, 7]

fig_contours, axs = decision_contours(d_list, m_list,
                            x_train, x_test, labels_train, labels_test,
                            tight_layout = 'pad', figsize=(20,20))

fig_contours.show()                            

### MNIST

In [13]:
(x_train, y_train), (x_test, y_test) = mnist.load_data('mnist.npz')
x_train, x_test = x_train.reshape((60000, 28, 28, 1)), x_test.reshape((10000, 28, 28, 1))

x_train, x_test = tf.convert_to_tensor(x_train), tf.convert_to_tensor(x_test)
y_train, y_test = tf.one_hot(y_train, 10), tf.one_hot(y_test, 10)
in_shape = x_train[0].shape

In [14]:
Dense = tf.keras.layers.Dense
Conv2D = tf.keras.layers.Conv2D
MaxPooling2D = tf.keras.layers.MaxPooling2D
Flatten = tf.keras.layers.Flatten


tn_MNIST = tf.keras.Sequential(
    [
     tf.keras.Input(shape=in_shape, batch_size=4096),
     Conv2D(28, (4,4), strides = (1,1), name='conv1'),
     MaxPooling2D((2,2), name='maxpool1'),
     Flatten(),
     MNIST_TN(bond_dim = 11),
     Dense(10, activation='softmax', name='out_layer')])
tn_MNIST.summary()

tn_MNIST.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['Precision'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (4096, 25, 25, 28)        476       
_________________________________________________________________
maxpool1 (MaxPooling2D)      (4096, 12, 12, 28)        0         
_________________________________________________________________
flatten_5 (Flatten)          (4096, 4032)              0         
_________________________________________________________________
mnist_tn_3 (MNIST_TN)        (4096, 256)               61504     
_________________________________________________________________
out_layer (Dense)            (4096, 10)                2570      
Total params: 64,550
Trainable params: 64,550
Non-trainable params: 0
_________________________________________________________________


In [15]:
check_point = ModelCheckpoint(filepath = os.getcwd()+'/tn_MNIST.hdf5', verbose = 0, save_best_only = True)

hist_dict = tn_MNIST.fit(x_train, y_train, batch_size = 16, epochs = 10, verbose = 0, validation_split = 0.2, shuffle=True, callbacks=[check_point])

TypeError: ('Not JSON Serializable:', <tf.Variable 'a:0' shape=(16, 12, 11) dtype=float32, numpy=
array([[[-0.0018468 ,  0.01204896,  0.01126017, ..., -0.00071777,
         -0.06228125,  0.04418238],
        [-0.03733819, -0.00878315,  0.06946413, ..., -0.05784283,
         -0.03232485, -0.03934333],
        [ 0.04812828, -0.05638238, -0.01038336, ..., -0.04579098,
          0.00557469,  0.01301471],
        ...,
        [-0.01516634,  0.01710937,  0.01890909, ...,  0.00128024,
         -0.02196125,  0.06371789],
        [-0.02712109,  0.03266293,  0.0326818 , ...,  0.01198524,
          0.01904359,  0.06642003],
        [-0.06266686, -0.01929544,  0.03905216, ...,  0.02441896,
         -0.01367769, -0.0507895 ]],

       [[ 0.0672924 ,  0.02509882,  0.00307088, ..., -0.01575541,
         -0.00784644,  0.01924504],
        [ 0.00252527,  0.02545468,  0.01987356, ..., -0.01439441,
          0.02339236,  0.02821402],
        [-0.03196016,  0.03390695, -0.02213433, ..., -0.07312908,
     

In [ ]:
tn_MNIST.input

In [ ]:
25*25*4096

In [ ]:
os.getcwd()

In [ ]:
MNIST_TN.get_config